# Daedalus Coverage

## Introduction

## Data

### Geomagnetic Kp Indices
We used Kp indices from previous years as they are recorded from NOAA web site:
* Explanation: https://www.ngdc.noaa.gov/stp/GEOMAG/kp_ap.html
* Download location: ftp://ftp.ngdc.noaa.gov/STP/GEOMAGNETIC_DATA/INDICES/KP_AP
There are available data for the years 1932-2018. The user can select which of these years' values to use for calculating the coverage for Daedalus.


![Solar Cycle](SolarCycle.png)

## Pseudocode

In [11]:
import sys
sys.path.insert(1, '../../SourceCode/')
import DaedalusGlobals as DaedalusGlobals

from os import path

import csv
import glob
import time
import datetime
import numpy as np 
import ipywidgets as w

import plotly
import chart_studio.plotly as py 
import plotly.graph_objects as go
import matplotlib.cm

# Holds the Geomagnetic kp Indices. 
# Accessing examples: GeomagneticIndices[("23", "05", "2011", "0")] stores the kp index for 23-05-2001 00:00-03:00
#                     GeomagneticIndices[("23", "05", "2011", "3")] stores the kp index for 23-05-2001 09:00-12:00
GeomagneticIndices = dict()

# GUI elements with global scope
style1 = {'description_width':'170px'}
layout1 = {'width':'780px'}
ExecutionTitle_Text = w.Text(value="", description='Execution title:', style=style1, layout=layout1)
ExecutionDescr_Text = w.Text(value="", description='Execution description:', style=style1, layout=layout1)
OrbitFilenames_Dropdown = w.Dropdown( options=glob.glob( DaedalusGlobals.Orbit_Files_Path + "DAED_ORB_Lifetime*.csv" ), description='Orbit files: ', style=style1, layout=layout1)
Years_Dropdown = w.Dropdown( value=2011, options=range(1932,2016),  description='Use Kp indices from year:', style=style1, layout=layout1)
SavedFilenames_Dropdown = w.Dropdown( options=glob.glob( DaedalusGlobals.CoverageResults_Files_Path + "*.CoverageResults.txt" ),  description='', style=style1, layout=layout1)


# utility: converts a number to its 2-digit string representation
def num_to_2digit_str( n ):
    s = str(n)
    if len(s) == 1:
        s = '0' + s
    return s

# utility: returns a color of a colormap as list of r,g,b,a values representing a value inside a range
def getColor( Value, minValue, maxValue, ColormapName ):
    cmap = matplotlib.cm.get_cmap( ColormapName )
    norm = matplotlib.colors.Normalize(vmin=minValue, vmax=maxValue)
    rgba = cmap( norm(Value) )
    s = "rgba" + str(rgba) 
    return s

# Define a class which can describe a bin
class Bin:
    ID             = ""
    Description    = ""
    MLT_min        = 0 # Mean Local Time (hour & min of the 24-hour day) (string)
    MLT_max        = 0 # Mean Local Time (hour & min of the 24-hour day) (string)
    MagLat_min     = 0 # Magnetic Latitude (degrees)
    MagLat_max     = 0 # Magnetic Latitude (degrees)
    Altitude_min   = 0 # Satellite's Altitude measured from Earth's surface (km)
    Altitude_max   = 0 # Satellite's Altitude measured from Earth's surface (km)
    Kp_min         = 0 #
    Kp_max         = 0 #
    NumOfBins      = 0 # How many parts will the Altitude range be splitted in
    CumulativeTime = 0 # (sec)
    DesirableCumulativeTime = 0 # (sec)
    
    def __init__(self, ID, Description, MLT_min, MLT_max, MagLat_min, MagLat_max, Altitude_min, Altitude_max, Kp_min, Kp_max, DesirableCumulativeTime):
        self.ID             = ID
        self.Description    = Description
        self.MLT_min        = MLT_min 
        self.MLT_max        = MLT_max
        self.MagLat_min     = MagLat_min
        self.MagLat_max     = MagLat_max
        self.Altitude_min   = Altitude_min
        self.Altitude_max   = Altitude_max
        self.Kp_min         = Kp_min
        self.Kp_max         = Kp_max
        self.DesirableCumulativeTime = DesirableCumulativeTime
        
    def getInfo(self):
        s  = self.ID.ljust(8, ' ') + ": "
        s += "%02.0f"%(self.MLT_min)      + "<MLT<="    + "%02.0f"%(self.MLT_max)      + " "
        s += "%03.0f"%(self.MagLat_min)   + "<MagLat<=" + "%03.0f"%(self.MagLat_max)   + " "
        s += "%03.0f"%(self.Altitude_min) + "<Alt<="    + "%03.0f"%(self.Altitude_max) + " "
        s += str(self.Kp_min)             + "<Kp<="     + str(self.Kp_max)       + " "
        s += "Coverage=" + "%06.0f"%(self.CumulativeTime/60) + "/" + "%03.0f"%(self.DesirableCumulativeTime/60) + "min"
        return s
    
    def printMe(self):
        print( self.getInfo() )

# this list holds the definitions of all bins
Bins = list()
#                ID        Description                          MLT      MagLat    Altitude                Kp       DesiredTime(sec)
Bins.append( Bin("AEM_L1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115, 120,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_L2", "Auroral E region, midnight sector", 22, 2,   60, 75,   120, 125,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_L3", "Auroral E region, midnight sector", 22, 2,   60, 75,   125, 130,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_L4", "Auroral E region, midnight sector", 22, 2,   60, 75,   130, 135,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_L5", "Auroral E region, midnight sector", 22, 2,   60, 75,   135, 140,               0, 2,   50*60 ) )
Bins.append( Bin("AEM_M1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115,       123.33333,   2, 4,   30*60 ) )
Bins.append( Bin("AEM_M2", "Auroral E region, midnight sector", 22, 2,   60, 75,   123.33333, 131.66666,   2, 4,   30*60 ) )
Bins.append( Bin("AEM_M3", "Auroral E region, midnight sector", 22, 2,   60, 75,   131.66666, 140,         2, 4,   30*60 ) )
Bins.append( Bin("AEM_H1", "Auroral E region, midnight sector", 22, 2,   60, 75,   115, 140,               4, 9,   20*60 ) )

Bins.append( Bin("AFM_L1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 185,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L2", "Auroral F region, midnight sector", 22, 2,   60, 75,   185, 230,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L3", "Auroral F region, midnight sector", 22, 2,   60, 75,   230, 275,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L4", "Auroral F region, midnight sector", 22, 2,   60, 75,   275, 320,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L5", "Auroral F region, midnight sector", 22, 2,   60, 75,   320, 365,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L6", "Auroral F region, midnight sector", 22, 2,   60, 75,   365, 410,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L7", "Auroral F region, midnight sector", 22, 2,   60, 75,   410, 455,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_L8", "Auroral F region, midnight sector", 22, 2,   60, 75,   455, 500,               0, 2,   50*60 ) )
Bins.append( Bin("AFM_M1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 230,               2, 4,   30*60 ) )
Bins.append( Bin("AFM_M2", "Auroral F region, midnight sector", 22, 2,   60, 75,   230, 320,               2, 4,   30*60 ) )
Bins.append( Bin("AFM_M3", "Auroral F region, midnight sector", 22, 2,   60, 75,   320, 410,               2, 4,   30*60 ) )
Bins.append( Bin("AFM_M4", "Auroral F region, midnight sector", 22, 2,   60, 75,   410, 500,               2, 4,   30*60 ) )
Bins.append( Bin("AFM_H1", "Auroral F region, midnight sector", 22, 2,   60, 75,   140, 260,               4, 9,   20*60 ) )
Bins.append( Bin("AFM_H2", "Auroral F region, midnight sector", 22, 2,   60, 75,   260, 380,               4, 9,   20*60 ) )
Bins.append( Bin("AFM_H3", "Auroral F region, midnight sector", 22, 2,   60, 75,   380, 500,               4, 9,   20*60 ) )

Bins.append( Bin("AEE_L1", "Auroral E region, evening sector",  16, 20,  60, 75,   115, 120,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_L2", "Auroral E region, evening sector",  16, 20,  60, 75,   120, 125,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_L3", "Auroral E region, evening sector",  16, 20,  60, 75,   125, 130,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_L4", "Auroral E region, evening sector",  16, 20,  60, 75,   130, 135,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_L5", "Auroral E region, evening sector",  16, 20,  60, 75,   135, 140,               0, 2,   50*60 ) )
Bins.append( Bin("AEE_M1", "Auroral E region, evening sector",  16, 20,  60, 75,   115,       123.33333,   2, 4,   30*60 ) )
Bins.append( Bin("AEE_M2", "Auroral E region, evening sector",  16, 20,  60, 75,   123.33333, 131.66666,   2, 4,   30*60 ) )
Bins.append( Bin("AEE_M3", "Auroral E region, evening sector",  16, 20,  60, 75,   131.66666, 140,         2, 4,   30*60 ) )
Bins.append( Bin("AEE_H1", "Auroral E region, evening sector",  16, 20,  60, 75,   115, 140,               4, 9,   20*60 ) )

Bins.append( Bin("AED_L1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115, 120,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_L2", "Auroral E region, dawn sector",     4, 8,  60, 75,   120, 125,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_L3", "Auroral E region, dawn sector",     4, 8,  60, 75,   125, 130,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_L4", "Auroral E region, dawn sector",     4, 8,  60, 75,   130, 135,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_L5", "Auroral E region, dawn sector",     4, 8,  60, 75,   135, 140,                 0, 2,   50*60 ) )
Bins.append( Bin("AED_M1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115,       123.33333,     2, 4,   30*60 ) )
Bins.append( Bin("AED_M2", "Auroral E region, dawn sector",     4, 8,  60, 75,   123.33333, 131.66666,     2, 4,   30*60 ) )
Bins.append( Bin("AED_M3", "Auroral E region, dawn sector",     4, 8,  60, 75,   131.66666, 140,           2, 4,   30*60 ) )
Bins.append( Bin("AED_H1", "Auroral E region, dawn sector",     4, 8,  60, 75,   115, 140,                 4, 9,   20*60 ) )

Bins.append( Bin("EEJ_A1", "Equatorial E-region",             10, 13,  -7,  7,   115,       123.33333,     0, 9,   10*60 ) )
Bins.append( Bin("EEJ_A2", "Equatorial E-region",             10, 13,  -7,  7,   123.33333, 131.66666,     0, 9,   10*60 ) )
Bins.append( Bin("EEJ_A3", "Equatorial E-region",             10, 13,  -7,  7,   131.66666, 140,           0, 9,   10*60 ) )

Bins.append( Bin("EPB_A1", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   140, 185,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A2", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   185, 230,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A3", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   230, 275,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A4", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   275, 320,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A5", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   320, 365,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A6", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   365, 410,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A7", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   410, 455,                  0, 9,   150*60 ) )
Bins.append( Bin("EPB_A8", "Equatorial Plasma Bubbles",       18,  4, -30, 30,   455, 500,                  0, 9,   150*60 ) )

Bins.append( Bin("SQ_A1",  "Sq & midlat F region currents",    6, 19, -60, 60,   140, 185,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A2",  "Sq & midlat F region currents",    6, 19, -60, 60,   185, 230,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A3",  "Sq & midlat F region currents",    6, 19, -60, 60,   230, 275,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A4",  "Sq & midlat F region currents",    6, 19, -60, 60,   275, 320,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A5",  "Sq & midlat F region currents",    6, 19, -60, 60,   320, 365,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A6",  "Sq & midlat F region currents",    6, 19, -60, 60,   365, 410,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A7",  "Sq & midlat F region currents",    6, 19, -60, 60,   410, 455,                  0, 3,   150*60 ) )
Bins.append( Bin("SQ_A8",  "Sq & midlat F region currents",    6, 19, -60, 60,   455, 500,                  0, 3,   150*60 ) )

Bins.append( Bin("CF_L1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 185,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L2", "Dayside Cusp F-region",            10, 14,   70,  80,   185, 230,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L3", "Dayside Cusp F-region",            10, 14,   70,  80,   230, 275,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L4", "Dayside Cusp F-region",            10, 14,   70,  80,   275, 320,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L5", "Dayside Cusp F-region",            10, 14,   70,  80,   320, 365,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L6", "Dayside Cusp F-region",            10, 14,   70,  80,   365, 410,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L7", "Dayside Cusp F-region",            10, 14,   70,  80,   410, 455,                0, 2,   50*60 ) )
Bins.append( Bin("CF_L8", "Dayside Cusp F-region",            10, 14,   70,  80,   455, 500,                0, 2,   50*60 ) )
Bins.append( Bin("CF_M1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 230,               2, 4,   30*60 ) )
Bins.append( Bin("CF_M2", "Dayside Cusp F-region",            10, 14,   70,  80,   230, 320,               2, 4,   30*60 ) )
Bins.append( Bin("CF_M3", "Dayside Cusp F-region",            10, 14,   70,  80,   320, 410,               2, 4,   30*60 ) )
Bins.append( Bin("CF_M4", "Dayside Cusp F-region",            10, 14,   70,  80,   410, 500,               2, 4,   30*60 ) )
Bins.append( Bin("CF_H1", "Dayside Cusp F-region",            10, 14,   70,  80,   140, 260,               4, 9,   20*60 ) )
Bins.append( Bin("CF_H2", "Dayside Cusp F-region",            10, 14,   70,  80,   260, 380,               4, 9,   20*60 ) )
Bins.append( Bin("CF_H3", "Dayside Cusp F-region",            10, 14,   70,  80,   380, 500,               4, 9,   20*60 ) )

Bins.append( Bin("PCF_L1", "Polar cap F-region",              14, 10,   70,  90,   140, 185,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L2", "Polar cap F-region",              14, 10,   70,  90,   185, 230,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L3", "Polar cap F-region",              14, 10,   70,  90,   230, 275,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L4", "Polar cap F-region",              14, 10,   70,  90,   275, 320,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L5", "Polar cap F-region",              14, 10,   70,  90,   320, 365,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L6", "Polar cap F-region",              14, 10,   70,  90,   365, 410,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L7", "Polar cap F-region",              14, 10,   70,  90,   410, 455,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_L8", "Polar cap F-region",              14, 10,   70,  90,   455, 500,               0, 2,   50*60 ) )
Bins.append( Bin("PCF_M1", "Polar cap F-region",              14, 10,   70,  90,   140, 230,               2, 4,   30*60 ) )
Bins.append( Bin("PCF_M2", "Polar cap F-region",              14, 10,   70,  90,   230, 320,               2, 4,   30*60 ) )
Bins.append( Bin("PCF_M3", "Polar cap F-region",              14, 10,   70,  90,   320, 410,               2, 4,   30*60 ) )
Bins.append( Bin("PCF_M4", "Polar cap F-region",              14, 10,   70,  90,   410, 500,               2, 4,   30*60 ) )
Bins.append( Bin("PCF_H1", "Polar cap F-region",              14, 10,   70,  90,   140, 260,               4, 9,   20*60 ) )
Bins.append( Bin("PCF_H2", "Polar cap F-region",              14, 10,   70,  90,   260, 380,               4, 9,   20*60 ) )
Bins.append( Bin("PCF_H3", "Polar cap F-region",              14, 10,   70,  90,   380, 500,               4, 9,   20*60 ) )



def createBinsForTheWholeEarth():
    n = 1
    for Kp_min in [0, 2, 4]:
        if Kp_min == 0:
            Kp_max = 2
        elif Kp_min == 2:
            Kp_max = 4
        elif Kp_min == 4:
            Kp_max = 9
        ####
        for MLT_min in range(0, 24, 4):
            for MagLat_min in range(-180, 180, 20):
                for Alt_min in range(115, 250, 25):
                    n = n + 1
                    Bins.append( Bin("E"+str(n), "",            MLT_min, MLT_min+4,   MagLat_min, MagLat_min+15,   Alt_min, Alt_min+25,               Kp_min, Kp_max,   20*60 ) )                    
    print ( len(Bins) + " Bins covering the whole Earth.")
    
#createBinsForTheWholeEarth()

# cheks if certain MLT lies in a certain range. Created in order to take account ranges like 22-2
def is_MLT_inside_range( MLT, MLT_min, MLT_max ):
    if MLT_min <= MLT_max: # example from 13 to 18 hour
        return (MLT > MLT_min  and  MLT <= MLT_max)
    else: # example from 22 to 3 hour
        return (MLT > MLT_min  or   MLT <= MLT_max)

# returns: the bin object which matches the arguments
def GetMatchedBin( MLT, MagLat, Altitude, Kp, Duration ):
    for B in Bins:
        Kp_min_to_check = B.Kp_min
        if Kp_min_to_check == 0:
            Kp_min_to_check = -1
        ####
        if is_MLT_inside_range(MLT, B.MLT_min, B.MLT_max):
            if         MagLat   > B.MagLat_min    and  MagLat   <= B.MagLat_max:
                if     Altitude > B.Altitude_min  and  Altitude <= B.Altitude_max:
                    if Kp       > Kp_min_to_check and  Kp       <= B.Kp_max:
                        B.CumulativeTime = B.CumulativeTime + Duration 
                        return B
    return B


# returns: the bin object which matches the arguments
def getBinByItsProperties( MLT_min, MLT_max, MagLat_min, MagLat_max, Altitude_min, Altitude_max, Kp_min, Kp_max ):
    CorrectBin = None
    for B in Bins:
        if             MLT_min      == B.MLT_min       and  MLT_max      == B.MLT_max:
            if         MagLat_min   == B.MagLat_min    and  MagLat_max   == B.MagLat_max:
                if     Altitude_min == B.Altitude_min  and  Altitude_max == B.Altitude_max:
                    if Kp_min       == B.Kp_min        and  Kp_max       == B.Kp_max:
                        CorrectBin = B
                        return CorrectBin
    return CorrectBin


# Reads the Geomagnetic kp Indices from text files and stores them in a dictionary.
# Data Source (explanation): https://www.ngdc.noaa.gov/stp/GEOMAG/kp_ap.html
# Data Source (download)   : ftp://ftp.ngdc.noaa.gov/STP/GEOMAGNETIC_DATA/INDICES/KP_AP
def readGeomagneticIndices():
    global GeomagneticIndices
    for Y in range(Years_Dropdown.value, Years_Dropdown.value+4): # this range should be small for execution speed
        with open(DaedalusGlobals.GeomagneticIndices_Files_Path + str(Y)) as fp:
            for line in fp:
                year  = "20" + line[0:2]
                month = line[2:4]
                day   = line[4:6]
                kp00  = float(line[12:14]) / 10
                kp03  = float(line[14:16]) / 10
                kp06  = float(line[16:18]) / 10
                kp09  = float(line[18:20]) / 10
                kp12  = float(line[20:22]) / 10
                kp15  = float(line[22:24]) / 10
                kp18  = float(line[24:26]) / 10
                kp21  = float(line[26:28]) / 10
                GeomagneticIndices[(day, month, year, "0")] = kp00
                GeomagneticIndices[(day, month, year, "1")] = kp03
                GeomagneticIndices[(day, month, year, "2")] = kp06
                GeomagneticIndices[(day, month, year, "3")] = kp09
                GeomagneticIndices[(day, month, year, "4")] = kp12
                GeomagneticIndices[(day, month, year, "5")] = kp15
                GeomagneticIndices[(day, month, year, "6")] = kp18
                GeomagneticIndices[(day, month, year, "7")] = kp21
            fp.close()
        
# returns: 1. how many bin misses occured
#          2. how many bin hits occured
#          3. the filename where the coverage results are stored
#          4. the duration of execution in seconds
def CalculateCoverage():
    Kp = 0
    startSecs = time.time()
    BinMisses = BinHits = 0
    PREV_time  = CURR_time  = None
    PREV_BinID = CURR_BinID = ""
    SelectedOrbit_OnlyFilename = OrbitFilenames_Dropdown.value[OrbitFilenames_Dropdown.value.rfind("/")+1:OrbitFilenames_Dropdown.value.rfind(".")]
    if path.exists( OrbitFilenames_Dropdown.value ):
        print( "File " + SelectedOrbit_OnlyFilename + " already exists. Cannot continue in order to prevent overwriting useful data." )
        return 0, 0, "", 0 # <<<<
    with open( OrbitFilenames_Dropdown.value ) as CSVfile:        
        CSVreader = csv.reader( CSVfile )
        # locate the column numnbers of interest inside the csv file
        CSVheader = next( CSVreader )
        Time_idx     = CSVheader.index( "Time (UTCG)" ) #CSVheader.index( "Daedalus.EpochText" )
        Lat_idx      = CSVheader.index( "Lat (deg)" ) #CSVheader.index( "Daedalus.Latitude" )
        Lon_idx      = CSVheader.index( "Lon (deg)" ) #CSVheader.index( "Daedalus.Longitude" )
        Altitude_idx = CSVheader.index( "Alt (km)" ) #CSVheader.index( "Daedalus.Height" )
        MagLat_idx   = CSVheader.index( "Daedalus.Magnetic Latitude" )
        MLT_idx      = CSVheader.index( "Daedalus.MLT" )
        # read the satellite positions and try to fill the bins
        n = 0
        for row in CSVreader: # for each satellite position
            n = n + 1
            PREV_time = CURR_time
            # find out the date-time of this satellite position
            CURR_time = datetime.datetime.strptime(row[Time_idx], '%d %b %Y %H:%M:%S.%f')
            year  = CURR_time.year
            month = CURR_time.month
            day   = CURR_time.day
            hour  = CURR_time.hour
            # calculate the Kp index for this particular time
            try:
                Kp = GeomagneticIndices[(num_to_2digit_str(day), num_to_2digit_str(month), num_to_2digit_str(Years_Dropdown.value), str(hour//3))]
            except:
                if month==2 and day==29: # the leap years may correspond to non-leap years at the selected range of years for Kp calculation
                    Kp = GeomagneticIndices[(num_to_2digit_str(28), num_to_2digit_str(month), num_to_2digit_str(Years_Dropdown.value), str(hour//3))]
            # find out some useful properties of this satellite position
            MLT      = float( row[MLT_idx] )
            MagLat   = float( row[MagLat_idx] )
            Altitude = float( row[Altitude_idx] )
            # Check if the satellite position can be assigned to a bin
            mathedBin = GetMatchedBin( MLT, MagLat, Altitude, Kp, 0 )
            PREV_BinID = CURR_BinID
            CURR_BinID = mathedBin.ID
            # If the satellite is inside a bin then calculate the duration
            if len(CURR_BinID) > 0  and  CURR_BinID == PREV_BinID:
                BinHits = BinHits + 1
                DurationInsideBin = (CURR_time - PREV_time).seconds
                mathedBin.CumulativeTime = mathedBin.CumulativeTime + DurationInsideBin 
            elif len(CURR_BinID) > 0:
                BinMisses = BinMisses + 1
    # calculate duration of execution
    finishSecs = time.time()
    DurationOfExecution = finishSecs-startSecs
    # Save the results in a text file
    nowstr = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    filename = DaedalusGlobals.CoverageResults_Files_Path + SelectedOrbit_OnlyFilename + ".CoverageResults.txt"
    F = open(filename, 'w')
    F.write( "# -- COVERAGE RESULTS -- " + "\n"  )
    F.write( "# Date of execution: " + nowstr + "\n" )
    F.write( "# Title: " + ExecutionTitle_Text.value + "\n" )
    F.write( "# Description: " + ExecutionDescr_Text.value + "\n")
    F.write( "# Parsed " + str(n) + " lines from orbit file " + OrbitFilenames_Dropdown.value + "\n")
    F.write( "# Used Kp indices starting from year " + str(Years_Dropdown.value) + "\n")
    F.write( "# Bin Misses: " + str(BinMisses) + "     Bin Hits: " + str(BinHits) + "\n")
    F.write( "# Duration of execution: " + str(DurationOfExecution) + " seconds = " + str(DurationOfExecution/60) + " minutes" + "\n" )
    F.write( "# " + "\n")
    for B in Bins:
        F.write( B.getInfo() + "\n" )
    F.close()
    #
    return BinMisses, BinHits, filename, DurationOfExecution

        

#################### EVENT LISTENERS ###########################
def Exec_Btn_Clicked( b ):
    print( "Coverage Calculation started." )
    readGeomagneticIndices()
    BinMisses, BinHits, filename, Duration = CalculateCoverage()
    print( "Coverage Calculation finshed in " + str(Duration) + " seconds, with " + str(BinMisses) + " bin misses, " + str(BinHits) + " bin hits." )
    print( "RESULTS (stored in " + filename + "):" )
    for B in Bins:
        B.printMe()
    PlotBins_Rectangles()
def Load_Btn_Clicked( b ):
    if len(SavedFilenames_Dropdown.value) > 0:
        with open(SavedFilenames_Dropdown.value, 'r') as F:
            for line in F:
                if line[0:1] == '#': # this line contains a comment, print it as it is.
                    print ( line[1:len(line)-1] )
                else: # this line contains bin info, print it and store them in the correct bin.
                    print ( line[:len(line)-1] )
                    aBinID = line[:line.find(":")]
                    aCoverageValue = line[line.rfind("=")+1:line.rfind("/")]
                    for B in Bins:
                        if B.ID == aBinID:
                            B.CumulativeTime = aCoverageValue
        F.close()
        PlotBins_Rectangles()
################################################################
def createGUI():
    MainTab = w.Tab() # the top level visual element
    CalcCoveragePanel = w.VBox()
    LoadCoveragePanel = w.HBox()
    MainTab.children = [ CalcCoveragePanel,    LoadCoveragePanel ]
    MainTab.set_title(0, 'Calculate Coverage')
    MainTab.set_title(1, 'Load Coverage')
    ## 
    Exec_Btn = w.Button (description='Calculate Coverage',tooltip="Click here to calculate the Daedalus coverage results",)
    Exec_Btn.style.button_color = 'MediumTurquoise'
    Exec_Btn.on_click( Exec_Btn_Clicked )
    CalcCoveragePanel.children = [OrbitFilenames_Dropdown, ExecutionTitle_Text, ExecutionDescr_Text, Years_Dropdown, Exec_Btn ]
    ##
    Load_Btn = w.Button (description='Load Coverage from',tooltip="Click here to plot the Daedalus coverage results",)
    Load_Btn.style.button_color = 'YellowGreen'
    Load_Btn.on_click( Load_Btn_Clicked )
    LoadCoveragePanel.children += (Load_Btn,)
    LoadCoveragePanel.children += (SavedFilenames_Dropdown,)
    return MainTab
        
    


def PlotBins_Rectangles():
    All_MagLatRanges = list()
    for B in Bins:
        if [B.MagLat_min, B.MagLat_max] not in All_MagLatRanges:
            All_MagLatRanges.append( [B.MagLat_min, B.MagLat_max] )
    
    for aMagLatRange in All_MagLatRanges:
        # We will create one plot for each MagLatRange 
        fig = go.Figure()
        
        # find out all MLT ranges for this MagLat range
        All_MLTRanges = list()
        for B in Bins:
            if B.MagLat_min==aMagLatRange[0] and  B.MagLat_max==aMagLatRange[1]:
                if [B.MLT_min, B.MLT_max] not in All_MLTRanges:
                    All_MLTRanges.append( [B.MLT_min, B.MLT_max] )
        
        # find out all Kp ranges for this MagLat range and this MLTrange 
        ColumnIDX = 0
        for aMLTrange in All_MLTRanges:
            All_KpRanges = list()
            for B in Bins:
                if B.MagLat_min==aMagLatRange[0] and  B.MagLat_max==aMagLatRange[1]:
                    if B.MLT_min==aMLTrange[0] and  B.MLT_max==aMLTrange[1]:
                        if [B.Kp_min, B.Kp_max] not in All_KpRanges:
                            All_KpRanges.append( [B.Kp_min, B.Kp_max] )

            # find out all Altitude ranges for this MagLat range, this MLT range and this Kp range
            for aKpRange in All_KpRanges:
                ColumnIDX += 1
                All_AltitudeRanges = list()
                for B in Bins:
                    if B.MagLat_min==aMagLatRange[0] and  B.MagLat_max==aMagLatRange[1]:
                        if B.MLT_min==aMLTrange[0] and  B.MLT_max==aMLTrange[1]:
                            if B.Kp_min==aKpRange[0] and  B.Kp_max==aKpRange[1]:
                                All_AltitudeRanges.append( [B.Altitude_min, B.Altitude_max] )
                                    
                # plot
                group_title = ""
                bar_title = "Kp:" + str(aKpRange[0]) + "-" + str(aKpRange[1])
                for anAltitudeRange in All_AltitudeRanges:
                    #print( "Looking for", aMLTrange[0], aMLTrange[1], aMagLatRange[0], aMagLatRange[1], anAltitudeRange[0], anAltitudeRange[1], aKpRange[0], aKpRange[1])
                    currentBin = getBinByItsProperties( aMLTrange[0], aMLTrange[1], aMagLatRange[0], aMagLatRange[1], anAltitudeRange[0], anAltitudeRange[1], aKpRange[0], aKpRange[1] )
                    fig.add_shape(go.layout.Shape(
                        type="rect", xref="x", yref="y",
                        x0=ColumnIDX*55, y0=currentBin.Altitude_min, x1=(ColumnIDX+1)*55-10, y1=currentBin.Altitude_max,
                        #line=dict( color="RoyalBlue", width=3,), fillcolor="LightSkyBlue", opacity=0.5,
                        line=dict( color="RoyalBlue", width=3,), fillcolor=getColor(currentBin.CumulativeTime, 0, currentBin.DesirableCumulativeTime, "jet"), opacity=0.5,
                    ))             
                    #BinTitle = str(currentBin.CumulativeTime) +  " / " + str(currentBin.DesirableCumulativeTime)  # currentBin.ID + " " + str(currentBin.MLT_min) + "-" + str(currentBin.MLT_max) + " " + "Kp:" + str(currentBin.Kp_min) + "-" + str(currentBin.Kp_max)
                    BinTitle = currentBin.ID + " " + str(currentBin.MLT_min) + "-" + str(currentBin.MLT_max) + " " + "Kp:" + str(currentBin.Kp_min) + "-" + str(currentBin.Kp_max)
                    print (BinTitle + "  >>  " + str(currentBin.Altitude_min) + "::" + str(currentBin.Altitude_max) )
                    fig.add_trace(go.Scatter(x=[ColumnIDX*55],y=[currentBin.Altitude_min+10],text=[BinTitle],mode="text",))
            ColumnIDX += 1
        
        
        fig.update_xaxes(range=[0, ColumnIDX*55 + 20], showgrid=False)
        fig.update_yaxes(range=[110, 520])
        fig.update_layout( width=1000, height=1600, )
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot(fig) 
        break
    
    
display( createGUI() )



Coverage Calculation started.
File DAED_ORB_Lifetime_LLA_MLT_MLat_RAAN00_180s already exists. Cannot continue in order to prevent overwriting useful data.
Coverage Calculation finshed in 0 seconds, with 0 bin misses, 0 bin hits.
RESULTS (stored in ):
AEM_L1  : 22<MLT<=02 060<MagLat<=075 115<Alt<=120 0<Kp<=2 Coverage=000000/050min
AEM_L2  : 22<MLT<=02 060<MagLat<=075 120<Alt<=125 0<Kp<=2 Coverage=000000/050min
AEM_L3  : 22<MLT<=02 060<MagLat<=075 125<Alt<=130 0<Kp<=2 Coverage=000000/050min
AEM_L4  : 22<MLT<=02 060<MagLat<=075 130<Alt<=135 0<Kp<=2 Coverage=000000/050min
AEM_L5  : 22<MLT<=02 060<MagLat<=075 135<Alt<=140 0<Kp<=2 Coverage=000000/050min
AEM_M1  : 22<MLT<=02 060<MagLat<=075 115<Alt<=123 2<Kp<=4 Coverage=000000/030min
AEM_M2  : 22<MLT<=02 060<MagLat<=075 123<Alt<=132 2<Kp<=4 Coverage=000000/030min
AEM_M3  : 22<MLT<=02 060<MagLat<=075 132<Alt<=140 2<Kp<=4 Coverage=000000/030min
AEM_H1  : 22<MLT<=02 060<MagLat<=075 115<Alt<=140 4<Kp<=9 Coverage=000000/020min
AFM_L1  : 22<MLT<=02

AFM_H2 22-2 Kp:4-9  >>  260::380
AFM_H3 22-2 Kp:4-9  >>  380::500
AEE_L1 16-20 Kp:0-2  >>  115::120
AEE_L2 16-20 Kp:0-2  >>  120::125
AEE_L3 16-20 Kp:0-2  >>  125::130
AEE_L4 16-20 Kp:0-2  >>  130::135
AEE_L5 16-20 Kp:0-2  >>  135::140
AEE_M1 16-20 Kp:2-4  >>  115::123.33333
AEE_M2 16-20 Kp:2-4  >>  123.33333::131.66666
AEE_M3 16-20 Kp:2-4  >>  131.66666::140
AEE_H1 16-20 Kp:4-9  >>  115::140
AED_L1 4-8 Kp:0-2  >>  115::120
AED_L2 4-8 Kp:0-2  >>  120::125
AED_L3 4-8 Kp:0-2  >>  125::130
AED_L4 4-8 Kp:0-2  >>  130::135
AED_L5 4-8 Kp:0-2  >>  135::140
AED_M1 4-8 Kp:2-4  >>  115::123.33333
AED_M2 4-8 Kp:2-4  >>  123.33333::131.66666
AED_M3 4-8 Kp:2-4  >>  131.66666::140
AED_H1 4-8 Kp:4-9  >>  115::140


 -- COVERAGE RESULTS -- 
 Date of execution: 03-02-2020_22-15-36
 Title: 180
 Description: 
 Parsed 634937 lines from orbit file /home/NAS/Data_Files/OrbitData/CSV_Data_Lifetime_Daedalus_LLA_180s.csv
 Used Kp indices starting from year 2011
 Bin Misses: 72710     Bin Hits: 11847
 Duration of execution: 26.159502267837524 seconds = 0.43599170446395874 minutes
 
AEM_L1  : 22<MLT<=02 060<MagLat<=075 115<Alt<=120 0<Kp<=2 Coverage=000000/050min
AEM_L2  : 22<MLT<=02 060<MagLat<=075 120<Alt<=125 0<Kp<=2 Coverage=000000/050min
AEM_L3  : 22<MLT<=02 060<MagLat<=075 125<Alt<=130 0<Kp<=2 Coverage=000000/050min
AEM_L4  : 22<MLT<=02 060<MagLat<=075 130<Alt<=135 0<Kp<=2 Coverage=000003/050min
AEM_L5  : 22<MLT<=02 060<MagLat<=075 135<Alt<=140 0<Kp<=2 Coverage=000000/050min
AEM_M1  : 22<MLT<=02 060<MagLat<=075 115<Alt<=123 2<Kp<=4 Coverage=000000/030min
AEM_M2  : 22<MLT<=02 060<MagLat<=075 123<Alt<=132 2<Kp<=4 Coverage=000000/030min
AEM_M3  : 22<MLT<=02 060<MagLat<=075 132<Alt<=140 2<Kp<=4 Coverage=000

AFM_H1 22-2 Kp:4-9  >>  140::260
AFM_H2 22-2 Kp:4-9  >>  260::380
AFM_H3 22-2 Kp:4-9  >>  380::500
AEE_L1 16-20 Kp:0-2  >>  115::120
AEE_L2 16-20 Kp:0-2  >>  120::125
AEE_L3 16-20 Kp:0-2  >>  125::130
AEE_L4 16-20 Kp:0-2  >>  130::135
AEE_L5 16-20 Kp:0-2  >>  135::140
AEE_M1 16-20 Kp:2-4  >>  115::123.33333
AEE_M2 16-20 Kp:2-4  >>  123.33333::131.66666
AEE_M3 16-20 Kp:2-4  >>  131.66666::140
AEE_H1 16-20 Kp:4-9  >>  115::140
AED_L1 4-8 Kp:0-2  >>  115::120
AED_L2 4-8 Kp:0-2  >>  120::125
AED_L3 4-8 Kp:0-2  >>  125::130
AED_L4 4-8 Kp:0-2  >>  130::135
AED_L5 4-8 Kp:0-2  >>  135::140
AED_M1 4-8 Kp:2-4  >>  115::123.33333
AED_M2 4-8 Kp:2-4  >>  123.33333::131.66666
AED_M3 4-8 Kp:2-4  >>  131.66666::140
AED_H1 4-8 Kp:4-9  >>  115::140
